In [7]:
%run "../../PyUtils/common.ipynb"

fileName="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_data_table";
dfOriginal = LoadDataSet(fileName, checkForDateTime=False);
dfOriginal[0:4]

<IPython.core.display.Javascript object>

,date,day,sdate,stime,edate,etime,durationhours,so2,co,o3,no2,pm10,tmax,amax,pmax,tmin,amin,pmin,tmode,amode,pmode,tmaxlesstmin,amaxlessamin,pmaxlesspmin,tatmeasurement,aatmeasurement,patmeasurement,season,pef,indexpef,extradate
0,2015-01-01,4,2014-12-31,18:11:00,2015-01-01,18:26:00,None,0.098,12.500,0.488,0.301,1770.000,-0.800,1021.500,58.000,-8.500,1014.800,30.000,-5.600,1019.100,38.000,7.700,6.700,28.000,-5.600,1021.300,44.000,4,290.000,Y,2015-01-01
1,2015-01-02,5,2015-01-01,18:26:00,2015-01-02,17:30:00,None,0.088,13.500,0.386,0.401,633.000,-0.800,1022.200,68.000,-8.300,1019.900,31.000,-7.300,1020.500,31.000,7.500,2.300,37.000,-2.500,1021.700,33.000,4,263.000,Y,2015-01-02
2,2015-01-03,6,2015-01-02,17:30:00,2015-01-03,18:15:00,None,0.173,19.200,0.229,0.719,968.000,1.900,1023.400,71.000,-9.600,1017.100,34.000,-7.400,1023.200,40.000,11.500,6.300,37.000,1.200,1017.100,60.000,4,265.000,Y,2015-01-03
3,2015-01-04,0,2015-01-03,18:15:00,2015-01-04,18:05:00,None,0.148,17.400,0.161,0.867,908.000,7.600,1017.100,76.000,0.200,1012.900,59.000,0.700,1013.600,65.000,7.400,4.200,17.000,3.400,1014.400,76.000,4,269.000,Y,2015-01-04


In [103]:
df=dfOriginal.copy()
df=df.fillna(0)
df.loc[df.stime == 'null', 'stime'] = "00:00:00"
df.loc[df.etime == 'null', 'etime'] = "00:00:00"
sdttm = df.sdate + " " + df.stime
edttm = df.edate + " " + df.etime
df.insert(0, 'sdttm', sdttm)
df.insert(1, 'edttm', edttm)

df.sdttm = pd.to_datetime(df.sdttm)
df.edttm = pd.to_datetime(df.edttm)

df.sort_values(by='sdttm', ascending=True, inplace=True)
df=df.drop(['date', 'sdate', 'stime', 'edate', 'etime', 'indexpef', 'extradate'], axis=1)
df.durationhours = (df.edttm - df.sdttm).apply(lambda x: x.total_seconds()/(60*60) )
df=df.reset_index(drop=True)

df = df[df.pef > 100]
pef = df.pef;

df=df.drop(['edttm', 'pef'], axis=1)
df.insert(1, 'pef1', pef)

pef[-10:]=0     # Set last 10 to zero for predictions
df.insert(1, 'pef', pef)

df.to_csv("HanD/hand1.csv", sep=';')
df

,sdttm,pef,pef1,day,durationhours,so2,co,o3,no2,pm10,tmax,amax,pmax,tmin,amin,pmin,tmode,amode,pmode,tmaxlesstmin,amaxlessamin,pmaxlesspmin,tatmeasurement,aatmeasurement,patmeasurement,season
0,2014-12-31 07:12:00,258.000,258.000,4,24.500,0.106,12.400,0.441,0.425,1362.000,2.500,1019.600,91.000,-8.400,1012.300,30.000,-2.400,1012.400,38.000,10.900,7.300,61.000,-8.400,1019.600,39.000,4
1,2014-12-31 18:11:00,290.000,290.000,4,24.250,0.098,12.500,0.488,0.301,1770.000,-0.800,1021.500,58.000,-8.500,1014.800,30.000,-5.600,1019.100,38.000,7.700,6.700,28.000,-5.600,1021.300,44.000,4
2,2015-01-01 07:42:00,279.000,279.000,5,22.700,0.079,12.900,0.436,0.298,1083.000,-3.900,1022.200,63.000,-8.500,1020.100,35.000,-7.300,1021.600,56.000,4.600,2.100,28.000,-6.800,1020.500,58.000,4
3,2015-01-01 18:26:00,263.000,263.000,5,23.067,0.088,13.500,0.386,0.401,633.000,-0.800,1022.200,68.000,-8.300,1019.900,31.000,-7.300,1020.500,31.000,7.500,2.300,37.000,-2.500,1021.700,33.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2015-12-29 06:16:00,0.000,265.000,3,24.633,0.186,23.900,0.075,1.103,1692.000,2.700,1030.900,77.000,-9.100,1027.200,38.000,-0.900,1027.200,40.000,11.800,3.700,39.000,-0.900,1027.200,57.000,4
727,2015-12-29 18:05:00,0.000,238.000,3,24.117,0.173,23.200,0.100,1.073,1714.000,5.700,1028.200,89.000,-2.900,1022.200,47.000,-0.900,1027.200,50.000,8.600,6.000,42.000,2.600,1023.100,89.000,4
728,2015-12-30 06:54:00,0.000,225.000,4,24.033,0.134,19.000,0.109,0.878,1414.000,5.700,1027.000,100.000,-1.900,1022.200,47.000,1.800,1022.600,100.000,7.600,4.800,53.000,1.300,1024.100,96.000,4
729,2015-12-30 18:12:00,0.000,225.000,4,23.917,0.135,18.100,0.160,0.754,1366.000,5.400,1028.100,100.000,1.000,1022.600,60.000,1.800,1023.000,100.000,4.400,5.500,40.000,1.900,1028.100,73.000,4


In [105]:
df.season.unique()


array([4, 1, 2, 3])